In [1]:
# make a cell print all the outputs instead of just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
from pathlib import Path

data_dir = Path("./data")
data_fpath = data_dir / "2022-07-bmvc-report-01-data2-adaptive-loss-01.csv"

import pandas as pd
df = pd.read_csv(data_fpath) 

def check_df(df):
    print(df.shape)
    print(df.dtypes)
    return df.head(5)

check_df(df)

(450, 12)
Name                                  object
supervise_mode                        object
normal_class_fullqualified            object
loss                                  object
loss_empirical_cdf_clip_threshold    float64
validate/roc-auc-histavg             float64
validate/avg-precision-histavg       float64
scheduler_parameters                  object
normal_class                           int64
test/roc-auc                         float64
test/avg-precision                   float64
mvtec_class_type                      object
dtype: object


,Name,supervise_mode,normal_class_fullqualified,loss,loss_empirical_cdf_clip_threshold,validate/roc-auc-histavg,validate/avg-precision-histavg,scheduler_parameters,normal_class,test/roc-auc,test/avg-precision,mvtec_class_type
0,mvtec.mvtec.cls14.it00,synthetic-anomaly-confetti,mvtec_14_zipper,pixelwise-batch-avg-clip-score-cdf-adaptive,0.05,0.890021,0.299138,[0.9985],14,0.874472,0.271487,object
1,mvtec.mvtec.cls14.it00,synthetic-anomaly-confetti,mvtec_14_zipper,pixelwise-batch-avg-clip-score-cdf-adaptive,0.15,0.874998,0.194596,[0.9985],14,0.951397,0.431241,object
2,mvtec.mvtec.cls14.it00,synthetic-anomaly-confetti,mvtec_14_zipper,pixelwise-batch-avg-clip-score-cdf-adaptive,0.15,0.835181,0.208224,[0.9985],14,0.938531,0.418613,object
3,mvtec.mvtec.cls14.it00,synthetic-anomaly-confetti,mvtec_14_zipper,pixelwise-batch-avg-clip-score-cdf-adaptive,0.05,0.885790,0.298539,[0.9985],14,0.873147,0.279015,object
4,mvtec.mvtec.cls14.it00,synthetic-anomaly-confetti,mvtec_14_zipper,pixelwise-batch-avg-clip-score-cdf-adaptive,0.05,0.903222,0.351732,[0.9985],14,0.883907,0.295785,object


In [5]:
data_df = pd.read_csv(data_fpath) 
df = data_df
df["scheduler_parameters"] = df["scheduler_parameters"].apply(lambda x: float(x.lstrip("[").rstrip("]")))
df["scheduler_parameters"] = df["scheduler_parameters"].apply(lambda x: {0.985000: "higher decay", 0.998500: "equiv. decay"}[x])
df["loss_empirical_cdf_clip_threshold"] = df["loss_empirical_cdf_clip_threshold"].apply(lambda x: str(x) if x else "n/a")
df.drop(columns=["Name"], inplace=True)
check_df(df)

(450, 11)
supervise_mode                        object
normal_class_fullqualified            object
loss                                  object
loss_empirical_cdf_clip_threshold     object
validate/roc-auc-histavg             float64
validate/avg-precision-histavg       float64
scheduler_parameters                  object
normal_class                           int64
test/roc-auc                         float64
test/avg-precision                   float64
mvtec_class_type                      object
dtype: object


,supervise_mode,normal_class_fullqualified,loss,loss_empirical_cdf_clip_threshold,validate/roc-auc-histavg,validate/avg-precision-histavg,scheduler_parameters,normal_class,test/roc-auc,test/avg-precision,mvtec_class_type
0,synthetic-anomaly-confetti,mvtec_14_zipper,pixelwise-batch-avg-clip-score-cdf-adaptive,0.05,0.890021,0.299138,equiv. decay,14,0.874472,0.271487,object
1,synthetic-anomaly-confetti,mvtec_14_zipper,pixelwise-batch-avg-clip-score-cdf-adaptive,0.15,0.874998,0.194596,equiv. decay,14,0.951397,0.431241,object
2,synthetic-anomaly-confetti,mvtec_14_zipper,pixelwise-batch-avg-clip-score-cdf-adaptive,0.15,0.835181,0.208224,equiv. decay,14,0.938531,0.418613,object
3,synthetic-anomaly-confetti,mvtec_14_zipper,pixelwise-batch-avg-clip-score-cdf-adaptive,0.05,0.885790,0.298539,equiv. decay,14,0.873147,0.279015,object
4,synthetic-anomaly-confetti,mvtec_14_zipper,pixelwise-batch-avg-clip-score-cdf-adaptive,0.05,0.903222,0.351732,equiv. decay,14,0.883907,0.295785,object


In [6]:
data_df = df

In [7]:
from pandas import DataFrame
import pandas as pd
import numpy as np
from scipy.stats import percentileofscore


def get_color_mask_compare_by_line_every_n_cols(pt, n):
        
    assert pt.shape[1] % n == 0, f"pt.shape[1] % n != 0: {pt.shape[1] % n}"

    def get_ordering_signal(subdf: DataFrame) -> DataFrame:
        """at each line, the values are assigned a value from 0 to 1 respecting the ordering, such that 0 is the min and 1 is the max"""
        # it's ok to round up to 3 decimals because i will probably never have more than 1001 values
        # and it makes the table easier to read/debug
        subdf = subdf.apply(
            lambda series: series.apply(lambda x: percentileofscore(np.sort(series.values), x, kind="strict") / 100), 
            axis=1
        )
        # 1/n is the min value of percentileofscore
        return (subdf / (1 / n)).round(decimals=0).astype(int)

    return np.concatenate(
        [   
            get_ordering_signal(pt.iloc[:, i * n:(i + 1) * n]).values
            for i in range(pt.shape[1] // n)
        ],
        axis=1,
    )
    
COLOR_0 = 'background-color: #DC143C55'
COLOR_MIDDLE = 'background-color: #FFFFFF55'
COLOR_1 = 'background-color: #228B2255'
    
def color_from_mask(color_mask: np.ndarray):
    n = color_mask.max() + 1
    print(f"n: {n}")
    assert n > 1, f"n > 1: {n}"
    
    if n == 2:
        def color(_):
            colormap = np.empty_like(color_mask, dtype=object)
            colormap[color_mask == 0] = COLOR_0
            colormap[color_mask == 1] = COLOR_1
            return colormap
        
        return color
    
    elif n == 3:
        def color(_):
            colormap = np.empty_like(color_mask, dtype=object)
            colormap[color_mask == 0] = COLOR_0
            colormap[color_mask == 1] = COLOR_MIDDLE
            colormap[color_mask == 2] = COLOR_1
            return colormap
        
        return color
    else:
        raise NotImplementedError(f"n > 3 is not implemented, found {n}")

In [8]:
from pandas import DataFrame
import pandas as pd
import numpy as np
from functools import partial

df = data_df.copy()

row_cols = ["mvtec_class_type", "normal_class_fullqualified"]
column_cols = ["supervise_mode", "loss", "loss_empirical_cdf_clip_threshold"]

metrics = {
    "test/avg-precision": ["mean", "std"],
    "validate/avg-precision-histavg": ["mean",],
    # "test/roc-auc": ["mean", "std"],
    # "validate/roc-auc-histavg": ["mean",],
}
metric_cols = [(k, v) for k, list_vals in metrics.items() for v in list_vals]

pt_all_metrics = pd.pivot_table(df, index=row_cols, columns=column_cols, aggfunc=metrics)

# when it's a percentage
pt_all_metrics = 100 * pt_all_metrics

pt_test_avg = pt_all_metrics[("test/avg-precision", "mean")]
# pt_test_avg = pt_all_metrics[("test/roc-auc", "mean")]
pt_test_avg.loc[("", "object mean"), :] = pt_test_avg.loc["object"].mean(axis=0)
pt_test_avg.loc[("", "texture mean"), :] = pt_test_avg.loc["texture"].mean(axis=0)
pt_test_avg.loc[("", "all mean"), :] = pt_test_avg.mean(axis=0)

pt_test_std = pt_all_metrics[("test/avg-precision", "std")]
# pt_test_std = pt_all_metrics[("test/roc-auc", "std")]
pt_test_std.loc[("", "object mean"), :] = pt_test_std.loc["object"].mean(axis=0)
pt_test_std.loc[("", "texture mean"), :] = pt_test_std.loc["texture"].mean(axis=0)
pt_test_std.loc[("", "all mean"), :] = pt_test_std.mean(axis=0)

pt_validate_histmean_avg = pt_all_metrics[("validate/avg-precision-histavg", "mean")]
# pt_validate_histmean_avg = pt_all_metrics[("validate/roc-auc-histavg", "mean")]
pt_validate_histmean_avg.loc[("", "object mean"), :] = pt_validate_histmean_avg.loc["object"].mean(axis=0)
pt_validate_histmean_avg.loc[("", "texture mean"), :] = pt_validate_histmean_avg.loc["texture"].mean(axis=0)
pt_validate_histmean_avg.loc[("", "all mean"), :] = pt_validate_histmean_avg.mean(axis=0)

pt_composed = pt_test_avg.applymap("{:.1f}".format) + " " + pt_test_std.applymap("({:.1f})".format) + " " + pt_validate_histmean_avg.applymap("[{:.1f}]".format)

/tmp/ipykernel_50071/1545779017.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pt_test_avg.loc[("", "object mean"), :] = pt_test_avg.loc["object"].mean(axis=0)
/tmp/ipykernel_50071/1545779017.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pt_test_std.loc[("", "object mean"), :] = pt_test_std.loc["object"].mean(axis=0)
/tmp/ipykernel_50071/1545779017.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pt_validate_histmean_a

# compare all

In [9]:
pt_test_avg


supervise_mode                              real-anomaly  \
loss                                            old-fcdd   
loss_empirical_cdf_clip_threshold                    nan   
mvtec_class_type normal_class_fullqualified                
object           mvtec_00_bottle               65.315374   
                 mvtec_01_cable                57.426476   
                 mvtec_02_capsule               7.735551   
                 mvtec_05_hazelnut             43.496970   
                 mvtec_07_metal_nut            86.071217   
                 mvtec_08_pill                 66.625924   
                 mvtec_09_screw                 3.415211   
                 mvtec_11_toothbrush           17.115199   
                 mvtec_12_transistor           50.826205   
                 mvtec_14_zipper               64.910375   
texture          mvtec_03_carpet               72.897424   
                 mvtec_04_grid                 16.372754   
                 mvtec_06_leather              38.871155   
                 mvtec_10_tile                 89.750323   
                 mvtec_13_wood                 53.468208   
                 object mean                   46.293850   
                 texture mean                  54.271973   
                 all mean                      49.109658   

supervise_mode                                                                           \
loss                                        pixelwise-batch-avg-clip-score-cdf-adaptive   
loss_empirical_cdf_clip_threshold                                                  0.05   
mvtec_class_type normal_class_fullqualified                                               
object           mvtec_00_bottle                                              67.252265   
                 mvtec_01_cable                                               57.788507   
                 mvtec_02_capsule                                             16.212461   
                 mvtec_05_hazelnut                                            43.062496   
                 mvtec_07_metal_nut                                           89.395604   
                 mvtec_08_pill                                                55.912012   
                 mvtec_09_screw                                                7.492276   
                 mvtec_11_toothbrush                                          14.930768   
                 mvtec_12_transistor                                          49.535695   
                 mvtec_14_zipper                                              52.486640   
texture          mvtec_03_carpet                                              68.263759   
                 mvtec_04_grid                                                17.234099   
                 mvtec_06_leather                                             36.274395   
                 mvtec_10_tile                                                85.858487   
                 mvtec_13_wood                                                47.562243   
                 object mean                                                  45.406872   
                 texture mean                                                 51.038597   
                 all mean                                                     47.394540   

supervise_mode                                          \
loss                                                     
loss_empirical_cdf_clip_threshold                 0.15   
mvtec_class_type normal_class_fullqualified              
object           mvtec_00_bottle             57.286663   
                 mvtec_01_cable              54.456499   
                 mvtec_02_capsule             7.336214   
                 mvtec_05_hazelnut           30.227097   
                 mvtec_07_metal_nut          85.401965   
                 mvtec_08_pill               59.466016   
                 mvtec_09_screw               1.411936   
                 mvtec_11_toothbrush  

In [10]:
pt = pt_composed.copy()
pt_color_from = pt_test_avg.copy()
color_mask = get_color_mask_compare_by_line_every_n_cols(pt_color_from, n=3)
pt.style.apply(color_from_mask(color_mask), axis=None)

n: 3
